# How to use the GaussianCountercharge Workchain

This notebook will explain the use of the `GaussianCountercharge` Workchain. 

Ths workchain implements a correction to the supercell energy based on an equivalent electrostatic model. Normally, this lower-level workchain need not be used directly as the higher-level `FormationEnergy` workchain will abstract the detail away and automate the generation of the necessary inputs. For completeness, and to give the option to use the correction directly, the use of the workchain is demonstrated below.

**NOTE!**
In this alpha version of `AiiDA-Defects` there are a number of features which are either not yet implemented or not well tested. These are listed below, with no guarentee of this being an exhaustive list.
Please bear these considerations in mind when testing the workchain.

* `epsilon`, the dielctric constant of the bulk host material, must be specified 'by hand'. Later this will be calculated from DFPT by the `FormationEnergy` workchain.
* Alignment of the electrostatic potentials is not yet automatic. Placeholder code will return 0.0 eV for these steps. The alignment should be done 'by hand'. A typical option is to take a planar average of the electrostatic potential along some axis, and then to align far from the defect. For cubic cells, a convenient option is to place point defects in the center of the box, and then take the alignment at the edge. 
* In principle, only cubic cells are currently supported, although for a large-enough supercell, it shouldn't matter. This would be interesting to prove/disprove. A change to any shape of supercell is on the TODO list.
* The width of the model gaussian is fixed currently, with a TODO for a routine to fit this. If one wants to play with this for testing, I can expose it as an input.




## Step-by-step Usage 
1. Setup AiiDA as normal

In [ ]:
# Get your normal profile
from aiida import load_profile
load_profile()

# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common
from aiida.plugins import WorkflowFactory

2. Import the workchain. Any issues with the import, such as the entry point not being found, would indicate an installation problem.

In [ ]:
gaussian_wc = WorkflowFactory('defects.formation_energy.corrections.gaussian_countercharge')
# Get a 'builder' object. This helps us to understand what input arguments are required
builder = gaussian_wc.get_builder()

3. Prepare the inputs for the workchain

In [ ]:
# Set the charge state of the defect of interest
builder.defect_charge = orm.Float(-2.)

# Three electroststic potentials are required:
# v_host : the electrostatic potential of the host supercell
# v_defect_q0 : the electrostatic potential of the defect supercell in the neutral charge state
# v_defect_q : the electrostatic potential of the defect supercell in the desired charge state
# These are used during potential alignment and so for now (in the alpha code) we can pass in placeholders

# Create an arbitrary array
placeholder_array = orm.ArrayData()
placeholder_array.set_array('test', np.ones(3))

# Assign it to the inputs
builder.v_host = placeholder_array
builder.v_defect_q0 = placeholder_array
builder.v_defect_q = placeholder_array

# Prepare a structre. Only the host structure is required as the user sets the defect location explicitly.
# Here, a dummy strucute data with no atoms is used, but any valid StructureData object can passed in. 
# Only information about the cell is used.
test_structure = orm.StructureData(cell=(np.eye(3)*10.))
builder.host_structure = test_structure

# Set the dielectric constant of the host material - remember that we must use AiiDA data types
builder.epsilon = orm.Float(5.76)

# Specify the postion of the defect site as a vector of crystal coordinates
builder.defect_site = orm.List(list=[0.5,0.5,0.5])

4. Submit the workchain

In [ ]:
workchain_future = engine.submit(builder)

5. Check the output

In [ ]:
if workchain_future.is_finished:
    if workchain_future.is_finished_ok:
        print('Finished successfully')
        # Print one of the output nodes. Others are available
        print('Electrostatic correction: {} Ha'.format(workchain_future.outputs.total_correction.value))
    else:
        print('Excepted')
else:
    print('Not yet finished')


## Optional Inputs

There are some optional inputs which control the behaviour of the electrostatic modelling.

### model_iterations_required
`model_iterations_required` sets the number of model supercells to compute. By default, 3 such models are computed.
These are one model with scale factor 1, i.e. the current size of input supercell, a model with scale factor 2, i.e. double the size of the current supercell (~8 times the volume), and one with a scale factor of 3 (triple the size, ~27 times larger volume). Increasing the number of iterations required increases the expense of the work chain by introducing larger and larger model supercells, but improves the quality of the fitting for the energy of the isolated supercell (i.e. an infinitiely large supercell giving the defect in the dilute limit). 
In principle, the default of 3 is already adequate for a decent fit, but this can and should be tested. 
In the future, the scaling may not be sequential as it is now, but may instead try to favour more 'spaced-out' supercells.

### cutoff
`cutoff` sets the planewave cutoff (in Rydberg) for the planewave basis used in the electrostatic model and sets the resolution of the grids used. 
The modelled interaction (q/r) is smooth and so good results are possible at the default of 40 Ry. In principle, increasing this value improves the accuracy of the results, in exchange for increased computational expense, but equally, the overall method has low sensitivity to this parameter.
